In [4]:
import os
import time
import torch
import pandas as pd
from PIL import Image

In [5]:
onehot_df = pd.read_csv("datasets/cropped_medium_one_hot.csv", index_col=0)
labels_df = onehot_df.copy()

In [6]:
one_hot_columns = onehot_df.columns.tolist()


def one_hot_to_class_list(row):
    return [col for col in one_hot_columns if row[col] == 1]


labels_df['labels'] = onehot_df.apply(
    one_hot_to_class_list, axis=1
)
labels_df = labels_df[['file_path', 'labels']]

In [8]:
from prompts.prompts import dynamic_prompt
from cog_vlm_batch import collate_fn, recur_move_to, device, tokenizer, model

image_folder = "/workspace/images"
from utils import SRC_DIR

# image_folder = SRC_DIR

data = labels_df.file_path.values.tolist()
length = len(data)
batch_size = 16

for idx in range(0, length, batch_size):
    input_sample_list = []
    start = time.time()

    for i in data[idx:idx + batch_size]:
        image = Image.open(os.path.join(image_folder, i)).convert('RGB')
        categories = labels_df[labels_df.file_path == i].values[0]
        query = dynamic_prompt(categories)
        input_sample = model.build_conversation_input_ids(
            tokenizer,
            query=dynamic_prompt(categories),
            history=[],
            images=[image],
            template_version='chat'
        )
        input_sample_list.append(input_sample)
    print(f"Prepare input time: {time.time() - start}")

    start = time.time()
    input_batch = collate_fn(input_sample_list, tokenizer)
    input_batch = recur_move_to(
        input_batch, device, lambda x: isinstance(x, torch.Tensor)
    )
    input_batch = recur_move_to(
        input_batch, torch.bfloat16, lambda x: isinstance(x, torch.Tensor) and torch.is_floating_point(x)
    )
    print(f"Prepare batch time: {time.time() - start}")

    gen_kwargs = {
        "max_new_tokens": 2048,
        "pad_token_id": 128002,
    }

    start = time.time()
    with torch.no_grad():
        outputs = model.generate(**input_batch, **gen_kwargs)
        outputs = outputs[:, input_batch['input_ids'].shape[1]:]
        outputs = tokenizer.batch_decode(outputs)

    outlist = [output.split("<|end_of_text|>")[0].strip() for output in outputs]
    print(outlist)
    print(f"Generate time: {time.time() - start}")

['secretary/vixen-clare-k-loses-leopard-blouse-before-posing-in-provocative-black-lingerie-95469791/cropped/95469791_119_a59d.jpg'
 list(['ass', 'brunette', 'clothed', 'heels', 'legs', 'lingerie', 'office', 'pantyhose', 'pawg', 'skirt', 'solo', 'undressing', 'uniform', 'upskirt'])]
['boots/sexy-brunette-zlata-removes-her-sexy-lingerie-in-a-garter-belt-and-nylons-30175061/cropped/30175061_010_f4e0.jpg'
 list(['ass', 'boots', 'brunette', 'clothed', 'heels', 'leather', 'legs', 'lingerie', 'mature', 'milf', 'panties', 'pawg', 'skinny', 'solo', 'stockings', 'tiny tits'])]
['socks/young-looking-white-girl-pulls-down-her-white-underwear-in-matching-socks-61586662/cropped/61586662_004_b431.jpg'
 list(['ass', 'asshole', 'clothed', 'legs', 'panties', 'pawg', 'pussy', 'shaved', 'socks', 'solo', 'spreading', 'teen', 'tiny tits', 'upskirt'])]
['non-nude/blonde-babe-with-a-nice-ass-jenni-p-unveils-her-adorable-titties-in-lingerie-57157285/cropped/57157285_098_3558.jpg'
 list(['armpit', 'ass', 'blond

KeyboardInterrupt: 